In [209]:
import re, nltk, spacy, gensim, os
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
from string import punctuation
import pandas as pd
import emoji
import re

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer



In [254]:
top_tags = []

def list_directories(path):
    """list files and directories in a given path"""
    arr = os.listdir(path)
    return arr


def clean_text(text):
    ''' Lowering text and removing undesirable marks
    '''
    
    text = text.lower()
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text) # matches all whitespace characters
    text = text.strip(' ')
    return text
    

token = ToktokTokenizer()
punct = punctuation
    
def strip_list_noempty(mylist):
    
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']
    
    
def clean_punct(text): 
    ''' Remove punctuations'''
    
    words = token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    
    for w in words:
        if w in top_tags:
            punctuation_filtered.append(w)
        else:
            w = re.sub('^[0-9]*', " ", w)
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))


stop_words = set(stopwords.words("english"))

def stopWordsRemove(text):
    ''' Removing all the english stop words from a corpus
    Parameter:
    text: corpus to remove stop words from it
    '''

    words = token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    
    return ' '.join(map(str, filtered))
    
    
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags, stop_words=stop_words):
    ''' It keeps the lemma of the words (lemma is the uninflected form of a word),
    and deletes the underired POS tags
    
    Parameters:
    
    texts (list): text to lemmatize
    allowed_postags (list): list of allowed postags, like NOUN, ADL, VERB, ADV
    '''
    lemma = wordnet.WordNetLemmatizer()       
    doc = nlp(texts) 
    texts_out = []
    
    for token in doc:
        
        if str(token) in top_tags:
            texts_out.append(str(token))
            
        elif token.pos_ in allowed_postags:
            
            if token.lemma_ not in ['-PRON-']:
                texts_out.append(token.lemma_)
                
            else:
                texts_out.append('')
     
    texts_out = ' '.join(texts_out)

    return texts_out
    
    
def strip_emoji(text):
#     print(emoji.emoji_count(text))
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text



def preprocess_text(df,column='description'):
    
    df[column] = df[column].apply(lambda x: clean_text(x))
    df[column] = df[column].apply(lambda x:  BeautifulSoup(x).get_text())
    df[column] = df[column].apply(lambda x: strip_emoji(x))

    df[column] = df[column].apply(lambda x: clean_punct(x)) 
    df[column] = df[column].apply(lambda x: stopWordsRemove(x)) 
    df[column] = df[column].apply(lambda x: lemmatization(x, ['NOUN', 'ADV']))

    return df

In [261]:

def generate_tags(df,column='description'):
    
    countVec =CountVectorizer()
    cv = countVec.fit_transform(df[column].str.lower())
    cv_feature_names = countVec.get_feature_names()
    feature_count = cv.toarray().sum(axis = 0)
    feature_count = sorted(dict(zip(cv_feature_names, feature_count)).items(), key=lambda item: item[1])[::-1][:25]

    return feature_count

In [262]:
files = list_directories('data')
files

['Accounting_25_records.csv',
 'Mathematics_173_records.csv',
 'Psychologist_75_records.csv',
 'Mechanical_Engineering_175_records.csv',
 'Pre-Law_&_Legal_Studies_169_records.csv']

In [263]:
category = []
for i in files:
    t = ''.join(i.split('_')[:-2])
    category.append(t)
category

['Accounting',
 'Mathematics',
 'Psychologist',
 'MechanicalEngineering',
 'Pre-Law&LegalStudies']

In [264]:
data = []
for cat, file in zip(category,files):
    df = pd.read_csv(f"data/{file}") 

    df = preprocess_text(df,column='description')
    tags1 = generate_tags(df,column='description')
    tags2 = generate_tags(df,column='job-title')
    
    data.append([ cat, file ,tags1,tags2 ])


In [265]:
print('###### title Based ########','\n')    
for j in data:
    print('\n',j[0],'----->',j[1],'\n')
    print(j[3],'\n')   

###### title Based ######## 


 Accounting -----> Accounting_25_records.csv 

[('accounting', 25), ('analyst', 5), ('assistant', 4), ('specialist', 3), ('associate', 3), ('senior', 2), ('of', 2), ('finance', 2), ('director', 2), ('corporate', 2), ('vp', 1), ('undergraduate', 1), ('reporting', 1), ('remote', 1), ('portfolio', 1), ('officer', 1), ('jr', 1), ('intern', 1), ('head', 1), ('global', 1), ('financial', 1), ('executive', 1), ('customer', 1), ('coordinator', 1), ('controller', 1)] 


 Mathematics -----> Mathematics_173_records.csv 

[('scientist', 75), ('data', 75), ('mathematics', 37), ('analyst', 18), ('associate', 14), ('applied', 14), ('research', 12), ('level', 12), ('entry', 12), ('remote', 10), ('tutor', 9), ('and', 9), ('science', 8), ('researcher', 7), ('quantitative', 7), ('intern', 7), ('in', 7), ('faculty', 6), ('math', 5), ('commercial', 5), ('teacher', 4), ('sr', 4), ('reviewer', 4), ('marketing', 4), ('assistant', 4)] 


 Psychologist -----> Psychologist_75_record

In [266]:
print('###### description Based ########','\n')    
for j in data:
    print('\n',j[0],'----->',j[1],'\n')
    print(j[2],'\n')    


###### description Based ######## 


 Accounting -----> Accounting_25_records.csv 

[('accounting', 125), ('team', 51), ('company', 51), ('finance', 44), ('experience', 42), ('process', 40), ('job', 38), ('work', 37), ('function', 36), ('ability', 36), ('employment', 35), ('type', 32), ('opportunity', 31), ('skill', 30), ('customer', 27), ('management', 26), ('service', 24), ('business', 24), ('year', 23), ('status', 22), ('project', 22), ('responsibility', 21), ('position', 21), ('level', 21), ('employee', 21)] 


 Mathematics -----> Mathematics_173_records.csv 

[('datum', 523), ('experience', 427), ('team', 291), ('work', 274), ('job', 269), ('science', 257), ('research', 247), ('employment', 243), ('technology', 238), ('level', 237), ('business', 234), ('student', 215), ('ability', 208), ('information', 207), ('opportunity', 203), ('industry', 194), ('skill', 189), ('model', 183), ('analysis', 175), ('product', 170), ('data', 163), ('knowledge', 161), ('type', 157), ('development',